In [ ]:
import pandas as pd, numpy as np, yfinance as yf, warnings
warnings.filterwarnings('ignore')

tickers = ['EURUSD=X', '^GSPC', 'GC=F']
data = yf.download(tickers, start='2015-01-01', end='2025-11-03')

# Use 'Adj Close' if available, otherwise fallback to 'Close'
if 'Adj Close' in data.columns:
    prices = data['Adj Close']
else:
    prices = data['Close']

prices.columns = ['EURUSD', 'SPX', 'GOLD']
rets = np.log(prices).diff().dropna()
print(rets.tail(3))


In [ ]:
# ================================================================
# 2. FACTOR DATA
# ================================================================

from pandas_datareader import data as pdr
from sklearn.decomposition import PCA

term = pdr.get_data_fred('T10Y2Y', start='2015-01-01', end='2025-11-03')
dxy  = pdr.get_data_fred('DTWEXBGS', start='2015-01-01', end='2025-11-03').pct_change()
pca = PCA(n_components=1)
pc1 = pd.DataFrame(pca.fit_transform(rets), index=rets.index, columns=['PC1'])

factors = pd.concat([term, dxy, pc1], axis=1).ffill()
factors.columns = ['TERM', 'DXY', 'PC1']
common_idx = rets.index.intersection(factors.index)
rets, factors = rets.loc[common_idx], factors.loc[common_idx]

In [ ]:
# ================================================================
# 3. REGRESS → BETA (Time-Series OLS per asset)
# ================================================================
import statsmodels.api as sm

def fit_beta(asset_ret, factor_df, window=252):
    y = asset_ret.iloc[-window:]
    X = sm.add_constant(factor_df.iloc[-window:])
    model = sm.OLS(y, X).fit()
    return model.params[1:], model.params[0], model.mse_resid  # beta, alpha, sigma_e^2

# Initialize beta matrix: rows = assets (n), columns = factors (k)
B = pd.DataFrame(index=rets.columns, columns=factors.columns)
alphas, idio_vars = {}, {}

# Loop over assets
for asset in rets.columns:
    beta, alpha, resid_var = fit_beta(rets[asset], factors)
    B.loc[asset] = beta  # Fill row for asset
    alphas[asset] = alpha * 252
    idio_vars[asset] = resid_var * 252

print("Beta Matrix B (n x k):\n", B.round(3))



In [88]:
# ================================================================
# 4. FACTOR MODEL COVARIANCE
# ================================================================
Omega_f = factors.cov().values * 252  # Annualize factor covariance
Psi = np.diag(list(idio_vars.values()))
cov_factor = B.values @ Omega_f @ B.values.T + Psi

In [89]:
# ================================================================
# 5. LEDOIT-WOLF + HYBRID COVARIANCE
# ================================================================
from sklearn.covariance import LedoitWolf

def lw_cov_60d(date, returns=rets):
    window = returns.loc[:date].tail(60)
    return pd.DataFrame(LedoitWolf().fit(window).covariance_,
                        index=rets.columns, columns=rets.columns)

cov_lw = lw_cov_60d(rets.index[-1])
cov_final = 0.7 * cov_lw + 0.3 * pd.DataFrame(cov_factor, index=rets.columns, columns=rets.columns)

In [ ]:
# ================================================================
# 6. BLACK-LITTERMAN + MV WEIGHTS
# ================================================================
risk_aversion = 1
n_assets = len(rets.columns)
mkt_w = pd.Series([1/n_assets]*n_assets, index=rets.columns)
pi = risk_aversion * cov_final @ mkt_w

P = np.array([[1, 0, -1],   # GOLD > EURUSD by 6%
              [0, 1,  0]])  # SPX = 9%
Q = np.array([0.06, 0.09])
tau = 0.05
Omega = np.diag([tau * (P[i] @ cov_final @ P[i].T) for i in range(P.shape[0])])

inv_tau_S = np.linalg.inv(tau * cov_final.values.astype(float))
bl_mu = np.linalg.inv(inv_tau_S + P.T @ np.linalg.inv(Omega) @ P) @ \
        (inv_tau_S @ pi.values + P.T @ np.linalg.inv(Omega) @ Q)
bl_mu = pd.Series(bl_mu, index=rets.columns)

# MV Optimization
import cvxpy as cp
w = cp.Variable(3)
ret = bl_mu.values @ w
risk = cp.quad_form(w, cov_final.values)
prob = cp.Problem(cp.Maximize(ret - risk_aversion * risk),
                  [cp.sum(w) == 1, w >= 0, w <= 0.6])
prob.solve()
w_opt = pd.Series(w.value.round(3), index=rets.columns)
print("Optimal Weights:\n", w_opt)

In [ ]:
# ================================================================
# 7. REBALANCE ±5%
# ================================================================
current_w = w_opt.copy()
def needs_rebalance(curr, target, thresh=0.05):
    return ((curr - target).abs() > thresh * target).any()

In [ ]:
# ================================================================
# 8. BACKTEST (Walk-Forward)
# ================================================================
dates = pd.date_range('2018-01-01', '2025-11-03', freq='B')
nav = pd.Series(index=dates, dtype=float); nav.iloc[0] = 1.0
current_w = pd.Series([1/n_assets]*n_assets, index=rets.columns)

for i, d in enumerate(dates[1:], 1):
    if d not in rets.index:
        nav[d] = nav.iloc[i-1]
        continue

    # Re-estimate everything rolling
    # (repeat steps 3–6 with .loc[:d])
    # For brevity: assume w_new computed
    if needs_rebalance(current_w, w_new):
        current_w = w_new
    daily_ret = (current_w * rets.loc[d]).sum()
    nav[d] = nav.iloc[i-1] * (1 + daily_ret)

nav = nav.ffill()
print(f"CAGR: {((nav[-1]/nav[0])**(252/len(nav))-1)*100:.1f}%")